# CS6140 Assignments

**Instructions**
1. In each assignment cell, look for the block:
 ```
  #BEGIN YOUR CODE
  raise NotImplementedError.new()
  #END YOUR CODE
 ```
1. Replace this block with your solution.
1. Test your solution by running the cells following your block (indicated by ##TEST##)
1. Click the "Validate" button above to validate the work.

**Notes**
* You may add other cells and functions as needed
* Keep all code in the same notebook
* In order to receive credit, code must "Validate" on the JupyterHub server

---

# Assignment 5: Model Evaluation and Regularization


In this assignment, we will investigate two evaluation methods and two ways that regularization can be used to control the behavior of linear models. Most of the code here will be copied or refactored from previous assignments. You are encouraged to copy **your code only** from previous assignments.

In [1]:
require './assignment_lib'

"if(window['d3'] === undefined ||\n   window['Nyaplot'] === undefined){\n    var path = {\"d3\":\"https://cdnjs.cloudflare.com/ajax/libs/d3/3.5.5/d3.min\",\"downloadable\":\"http://cdn.rawgit.com/domitry/d3-downloadable/master/d3-downloadable\"};\n\n\n\n    var shim = {\"d3\":{\"exports\":\"d3\"},\"downloadable\":{\"exports\":\"downloadable\"}};\n\n    require.config({paths: path, shim:shim});\n\n\nrequire(['d3'], function(d3){window['d3']=d3;console.log('finished loading d3');require(['downloadable'], function(downloadable){window['downloadable']=downloadable;console.log('finished loading downloadable');\n\n\tvar script = d3.select(\"head\")\n\t    .append(\"script\")\n\t    .attr(\"src\", \"http://cdn.rawgit.com/domitry/Nyaplotjs/master/release/nyaplot.js\")\n\t    .attr(\"async\", true);\n\n\tscript[0][0].onload = script[0][0].onreadystatechange = function(){\n\n\n\t    var event = document.createEvent(\"HTMLEvents\");\n\t    event.initEvent(\"load_nyaplot\",false,false);\n\t    win

true

# Question 1.1 (1 Point)

Reuse **YOUR** implementation of ```StochasticGradientDescent``` from a previous assignment.

In [2]:
class StochasticGradientDescent
  attr_reader :weights
  attr_reader :objective
  def initialize obj, w_0, lr = 0.01
    @objective = obj
    @weights = w_0
    @n = 1.0
    @lr = lr
  end
  def update x
    
    dw = @objective.grad(x, @weights)
    learning_rate = @lr/Math.sqrt(@n)
   
    dwSize = dw.size
    dwSize.times do |i|
      dwKey = dw.keys[i]
      unless weights[dwKey].nil?
        weights[dwKey] = weights[dwKey] - learning_rate * dw[dwKey]
      end
    end
   
    @n = @n + 1
  
    @weights=@objective.adjust(weights)
    
  def adjust w
    w.each do |key,array|
      w[key] = [[0.001, w[key]].max, 0.999].min
    end
    return w
  end
  

   
  end
end

:update

In [3]:
### Hidden Test ###
assert_not_nil(StochasticGradientDescent.class)

## Question 1.2 (1 point)

Copy **YOUR** implementation of the ```dot``` product and ```norm``` functions from a previous assignment into the following cell. Please copy the whole function, not just the parts within the comments.

In [4]:
def dot x, w
  sum=0
  x.each do |key1, array1|
    w.each do |key2, array2|
      if key1==key2 then
        sum+=array1*array2
      end
    end
  end
  return sum

end

def norm w
  sum=0
  sum = Math.sqrt(dot(w,w))
  return sum
end
  


:norm

In [5]:
### Hidden test ###


## Question 1.3 (1 Point)

Refactor **YOUR** $z$-score normalization method from a previous assignment, where it might have been called  ```create_zspambase```. It should be general enough to normalize any dataset. Only normalize features in the ```features``` key.

Note: Watch out for zero-stdev features.

In [6]:
def z_normalize dataset
  zdataset = dataset.clone
  zdataset["data"] = dataset["data"].collect do |r|
    u = r.clone
    u["features"] = r["features"].clone
    u
  end

  sums=Hash.new {|h,k| h[k] = 0.0}
  mean=Hash.new {|h,k| h[k] = 0.0}
  sigma=Hash.new {|h,k| h[k] = 0.0}
  total=Hash.new {|h,k| h[k] = 0.0}
  size=(zdataset["data"].size).to_f

  zdataset["data"].each do |item|
    item["features"].each do |key,array|
      sums[key]=array+sums[key]
      total[key]=total[key]+1
    end
  end
  
  sums.each do |key,item|
    mean[key]=item.to_f/(total[key].to_f)
  end

  zdataset["data"].each do |item|
    item["features"].each do |key,array|
      sigma[key]=(array.to_f-mean[key].to_f)**2+sigma[key].to_f
    end
  end
  
  sigma.each do |key,item|
    sigma[key]=(sigma[key].to_f/(total[key].to_f-1.0))**0.5
  end
  
  
  zdataset["data"].each do |item|
    item["features"].each do |key,array|
      if key=="bias"
        next
      else
        item["features"][key]=(array.to_f-mean[key].to_f)/sigma[key].to_f
      end
    end
  end

  return zdataset
end

:z_normalize

In [7]:
def mean(array)
  array = array.inject(0.0) { |sum, x| sum += x } / array.size.to_f
end

def standard_deviation(array)
  m = mean(array)
  variance = array.inject(0.0) { |variance, x| variance += (x - m) ** 2 } 
  standard_deviation = Math.sqrt(variance/(array.size-1).to_f)
  return standard_deviation
end

def z_normalize dataset
  
  puts dataset
  zdataset = dataset.clone
  avg = Hash.new {|h,k| h[k] = 0.0}
  dev = Hash.new {|h,k| h[k] = 0.0}
  
  data = dataset["data"]
  dataSize = data.size
  
  dataSize.times do |i|
    r = data[i]["features"]
    rSize = r.size
    rSize.times do |j|
      rKey = r.keys[j]
      if avg[rKey] == 0.0
        array = []
        dataSize.times do |k|
            unless data[k]["features"][rKey].nil?
              array.push(data[k]["features"][rKey])
            end
        end
        avg[rKey] = mean(array)
        dev[rKey] = standard_deviation(array)
        if dev[rKey] == 0.0
          dev[rKey] = 1
          avg[rKey] = avg[rKey] - 1
        end
      end
    end
  end
  
  
  zdataset["data"] = dataset["data"].collect do |r|
    u = r.clone    
    u["features"] = r["features"].clone
    uSize = u["features"].size
    uSize.times do |i|
      uKey = u["features"].keys[i]      
      u["features"][uKey] = (u["features"][uKey] - avg[uKey])/(dev[uKey])
    end
    u
  end

  return zdataset
  
end

:z_normalize

In [8]:
### TEST ###
def test_13()
  spambase = read_sparse_data_from_csv "spambase"
  zspambase = z_normalize spambase

  assert_in_delta 0.27, spambase["data"].first["features"]["word_freq_our"], 1e-5
  assert_in_delta -0.628106690674003, zspambase["data"].first["features"]["word_freq_our"], 1e-5

  assert_in_delta 607.0, spambase["data"].first["features"]["capital_run_length_total"], 1e-5
  assert_in_delta 0.53386, zspambase["data"].first["features"]["capital_run_length_total"], 1e-5
end

test_13()

{"classes"=>{}, "features"=>["word_freq_make", "word_freq_address", "word_freq_all", "word_freq_3d", "word_freq_our", "word_freq_over", "word_freq_remove", "word_freq_internet", "word_freq_order", "word_freq_mail", "word_freq_receive", "word_freq_will", "word_freq_people", "word_freq_report", "word_freq_addresses", "word_freq_free", "word_freq_business", "word_freq_email", "word_freq_you", "word_freq_credit", "word_freq_your", "word_freq_font", "word_freq_000", "word_freq_money", "word_freq_hp", "word_freq_hpl", "word_freq_george", "word_freq_650", "word_freq_lab", "word_freq_labs", "word_freq_telnet", "word_freq_857", "word_freq_data", "word_freq_415", "word_freq_85", "word_freq_technology", "word_freq_1999", "word_freq_parts", "word_freq_pm", "word_freq_direct", "word_freq_cs", "word_freq_meeting", "word_freq_original", "word_freq_project", "word_freq_re", "word_freq_edu", "word_freq_table", "word_freq_conference", "char_freq_;", "char_freq_(", "char_freq_[", "char_freq_!", "char_fre

## Question 2.1 (5 Points)

Change your ```LinearRegression``` implementation from to implement regularization. The new implementation requires a value for $\lambda$. The regularization objective function for linear regression in a mini-batch is as follows:

# $L(w,X) = \frac{\lambda}{2} \left\lVert w \right\rVert ^ 2 + \frac{1}{n} \sum_{i} \frac{1}{2} \left(f(w,x_i) - y_i\right) ^ 2$

where ```reg_param``` corresponds to $\lambda$ in the formula above.

Note that there is no $\frac{1}{n}$ in front of the regularizer penalty. The ```predict``` and ```adjust``` methods have been provided for you. 

Hint: Use ```dot``` and ```norm``` as needed.

In [9]:
class LinearRegressionModelL2
  attr_reader :reg_param
  def initialize reg_param
    @reg_param = reg_param
  end

  def predict row, w
    x = row["features"] 
    yhat = dot(x,w)
  end
  
  def adjust w
    w.each_key {|k| w[k] = 0.0 if w[k].nan? or w[k].infinite?}
    w.each_key {|k| w[k] = 0.0 if w[k].abs > 1e5 }
  end
  
  def func examples, w
    
    sum=0
    model2 = LinearRegressionModelL2.new 0.0
    

    examples.each do |key1, array1|
      sum+=0.5*((model2.predict(key1,w)-key1["label"])**2)
    end
    
    return (sum/examples.size)+@reg_param/2*(norm(w))**2

  end
end

:func

In [10]:
### TEST ###
def test_21()
  m = LinearRegressionModelL2.new 0.0
  w = {"x1" => 1.0, "x2" => -3.0}
  examples = [
      {"features" => {"x1" => 0.7, "x2" => -0.3}, "label" => 0.97},
      {"features" => {"x1" => -2.7, "x2" => -1.3}, "label" => -1.0}  
  ]
  dataset = {"data" => examples}
  
  e1 = 0.19845
  assert_in_delta e1, m.func(examples[0,1], w), 1e-2, "1"
  
  e2 = 2.42
  assert_in_delta e2, m.func(examples[1,1], w), 1e-2, "2"    
  assert_in_delta (e1 + e2) / 2.0, m.func(examples, w), 1e-2, "3"  

  assert_in_delta 3.1622776602, norm(w), 1e-2, "4"
  m2 = LinearRegressionModelL2.new 1.7
  assert_in_delta 9.809225, m2.func(examples, w), 1e-2, "5"
end

test_21()

## Question 2.2 (5 Points)

Implement the gradient for the regularized linear regression using the above objective function.

In [11]:
class LinearRegressionModelL2
  def grad examples, w
    
    gradient = Hash.new {|h,k| h[k] = 0.0}
    sum = Hash.new {|h,k| h[k] = 0.0}

    w.each do |key,array|
      if !((array.is_a? Float) or (array.is_a? Integer))
        w[key]=0.0
      end
    end
        

    examples.each do |key1, array1|
      key1["features"].each do |key2, array2|
        sum[key2]+=(predict(key1,w)-key1["label"])*key1["features"][key2]
      end
    end
    
    
    sum.each do |key2, array2|
      gradient[key2]=(sum[key2]/examples.size)+(@reg_param)*w[key2]
    end
    
    return gradient

  end
end

:grad

In [12]:
### TEST ###
def test_22_0()
  #First, test when there is no regularization term, @reg_param = 0.0
  m = LinearRegressionModelL2.new 0.0
  w = {"x1" => 1.0, "x2" => -3.0}
  examples = [
      {"features" => {"x1" => 0.7, "x2" => -0.3}, "label" => 0.97},
      {"features" => {"x1" => -2.7, "x2" => -1.3}, "label" => -1.0}  
  ]
  dataset = {"data" => examples}
  
  # Test on a single example
  expected_g1 = 0.441
  g1 = m.grad(examples[0,1], w)
  puts "g1", g1
  
  assert_in_delta expected_g1, g1["x1"], 1e-2, "1"
  
  # Test on another single example
  expected_g2 = -5.94
  g2 = m.grad(examples[1,1], w)
  puts "g2", g2
  assert_in_delta expected_g2, g2["x1"], 1e-2, "2"  
  
  # Test on both example, should be the average of the two previous gradients
  expected_g3 = (expected_g1 + expected_g2) / 2.0
  g3 = m.grad(examples, w)  
  puts "g3", g3
  assert_in_delta expected_g3, g3["x1"], 1e-2, "3"  
end

test_22_0()

g1
{"x1"=>0.4409999999999999, "x2"=>-0.18899999999999997}
g2
{"x1"=>-5.940000000000001, "x2"=>-2.8600000000000003}
g3
{"x1"=>-2.7495000000000007, "x2"=>-1.5245000000000002}


In [13]:
### TEST ###
def test_22_1()
  # Test when the regularization parameter is nonzero
  m2 = LinearRegressionModelL2.new 1.7
  w = {"x1" => 1.0, "x2" => -3.0}
  examples = [
      {"features" => {"x1" => 0.7, "x2" => -0.3}, "label" => 0.97},
      {"features" => {"x1" => -2.7, "x2" => -1.3}, "label" => -1.0}  
  ]
  dataset = {"data" => examples}

  #Compare this to g3 in the previous test
  g3_with_regularization = m2.grad(examples, w)
  puts "g3_with_regularization", g3_with_regularization  
  assert_in_delta -1.0495, g3_with_regularization["x1"], 1e-2, "1"
  
  #Use a different regularization penalty
  m3 = LinearRegressionModelL2.new 0.3
  g3_with_different_regularization = m3.grad(examples, w)
  puts "g3_with_different_regularization", g3_with_different_regularization  
  assert_in_delta -2.4495, g3_with_different_regularization["x1"], 1e-2, "1"
  
end

test_22_1()

g3_with_regularization
{"x1"=>-1.0495000000000008, "x2"=>-6.624499999999999}
g3_with_different_regularization
{"x1"=>-2.449500000000001, "x2"=>-2.4245}


## Question 2.3 (5 Points)

Implement a function that calculates the Root Mean Squared Error (RMSE) for a given dataset for a prediction model and weights.

RMSE is defined as follows:

# $e = \sqrt{\frac{\sum_{i=1}^N{ \left( \hat{y} - y \right) ^ 2 }}{N}}$

where $N$ is the number of examples in the dataset.

Hint: Use the ```mean``` function in the assignment library.

In [14]:
def score_regression_model_rmse(data, weights, model)

  sum=0
  data.each do |key, array|
    sum+=(model.predict(key, weights)-key["label"])**2
  end
  
  return (sum/data.size)**(0.5)

end

:score_regression_model_rmse

In [15]:
### TEST ###
def test_23()
  m = LinearRegressionModelL2.new 0.0
  w = {"x1" => 1.0, "x2" => -3.0}
  x = [
      {"features" => {"x1" => 0.7, "x2" => -0.3}, "label" => 0.97},
      {"features" => {"x1" => -2.7, "x2" => -1.3}, "label" => -1.0}  
  ]
  
  e1 = (((0.7 + -3 * -0.3) - 0.97) ** 2)
  e2 = (((-2.7 + -3.0 * -1.3) - -1.0) ** 2)
  
  rmse = Math.sqrt((e1 + e2) / 2.0)
  assert_in_delta rmse, score_regression_model_rmse(x, w, m), 1e-2, "3"  
end

test_23()

## Question 3.1 (10 Points)

Using a small provided dataset, shown below, we will investigate model complexity. First, implement a _polynomial_ feature representation. 

Call the bias feature "1" for this part. For a dataset with two features, $x_1$ and $x_2$, a polynomial representation of degree 0 is as follows:

# $\phi(x, k = 0) = \left( 1 \right)$

degree 1:

# $\phi(x, k = 1) = \left( 1, x_1, x_2 \right)$

degree 2: 

# $\phi(x, k = 2) = \left( 1, x_1, x_2, x_1^2, x_2^2, x_1 x_2  \right)$

and more generally, for degree $k$:

# $\phi(x, k) = \left(1, x_1 \phi(x,k-1), x_2 \phi(x,k-1) \right)$

For your convenience, the function ```poly_features``` emits, for degree $k$, the names of the features to be multiplied. After generating the features, apply ```z_normalize``` to only the newly added features (i.e., not the original features or the bias).

Note: You may notice that the dataset we plan to use only has one feature and therefore the above seems overly complex. Don't worry, we will see this again. ;)

In [16]:
polydata = read_sparse_data_from_csv "polydata"
x1 = polydata["data"].collect {|r| r["features"]["x1"]}
x2 = polydata["data"].collect {|r| r["label"]}
puts "Polydata Regression Dataset"
Daru::DataFrame.new({x1: x1, x2: x2})
.plot(type: :scatter, x: :x1, y: :x2) do |plot, diagram|
  plot.x_label "X1"
  plot.y_label "Label"
  plot.legend false
end

Polydata Regression Dataset


#<Nyaplot::Plot:0x000000000291f260 @properties={:diagrams=>[#<Nyaplot::Diagram:0x00000000028e7a68 @properties={:type=>:scatter, :options=>{:x=>:x1, :y=>:x2}, :data=>"9ea043f0-9f19-4f62-bffd-904484ddea76"}, @xrange=[5.62850407100423, 34.2002350155681], @yrange=[-2.52858273819375, 4.60878015052933]>], :options=>{:x_label=>"X1", :y_label=>"Label", :legend=>false, :zoom=>true, :width=>700, :xrange=>[5.62850407100423, 34.2002350155681], :yrange=>[-2.52858273819375, 4.60878015052933]}}>

### Polynomial features

The ```poly_features``` function below returns a set of strings which represents all the possible features for a polynomial feature of degree $k$. In the example above, 

```ruby
poly_features ["x1", "x2"], 3
```

returns all polynomial features up to degree 3, which are as follows:

```["1", "x1", "x2", "1*x1*x1", "1*x1*x2", "1*x2*x2", "1*x1*x1*x1", "1*x1*x1*x2", "1*x1*x2*x2", "1*x2*x2*x2"]```

When provided a ```dataset```, the function ```create_polynomial_features``` you will implement below adds the new features (if they do not already exist) to the ```dataset["features"]``` top-level array. Then, for each example, apply the transformation. This will require manipulating the string and implementing a parser. For example, if the provided feature name is ```"1*x1*x1*x2"```, you will need to calculate $x_1^2 x_2$ as a new feature. Note that each original feature is provided separated by the ```*``` character. Use the ```split``` function for the string.

In [17]:
def poly_features features, degree
  poly_features = ["1"]

  degree.times do |i|
    poly_features += poly_features.flat_map do |x_prev|
      features.reject {|x| x == "1" or x == "bias"}.collect do |x|
        [x, x_prev.split("*")].flatten.sort.join("*")
      end
    end
    poly_features.uniq!
  end
  poly_features.collect {|k| k.gsub /^1\*([^\*]*)$/, '\1'}
end

poly_features ["x1", "x2"], 3

["1", "x1", "x2", "1*x1*x1", "1*x1*x2", "1*x2*x2", "1*x1*x1*x1", "1*x1*x1*x2", "1*x1*x2*x2", "1*x2*x2*x2"]

In [1]:
def create_polynomial_features dataset, degree
  polydataset = dataset.clone
  features = poly_features dataset["features"], degree

  array1=[]
  dataset1=dataset.clone
  
  dataset1["data"].each do |key,array|
    features.each do |item|
      a=[]
      answer=1
      a+=item.split("*")
      a.each do |item2|
        answer=key["features"][item2]*answer
      end
      key["features"][item]= answer
    end
  end

  dataset1=z_normalize(dataset1)
  
  i=0
  dataset1["data"].each do |key,array|
    features.each do |item|
      a=[]
      answer=1
      a+=item.split("*")
      a.each do |item2|
        if a.size==1
            key["features"][a[0]]=dataset["data"][i]["features"][a[0]]
        end
      end
    end
  end
  return dataset1

end

:create_polynomial_features

In [19]:
def test_31()
  data = read_sparse_data_from_csv "polydata"
  assert_in_delta 12.8132, data["data"].first["features"]["x1"], 1e-2, "1"
  
  polydata = create_polynomial_features data, 3
  
  xp = polydata["data"].first["features"]
  assert_in_delta 12.8132, xp["x1"], 1e-2, "2: Does not normalize original features"
  assert_in_delta -0.905, xp["1*x1*x1"], 1e-2, "3: Applies normalization to new features"
  assert_in_delta -0.827, xp["1*x1*x1*x1"], 1e-2, "4: Applies normalization to new features"
end

test_31()

{"classes"=>{}, "features"=>["x1", "1"], "data"=>[{"features"=>{"x1"=>12.8132597900148, "1"=>1, "1*x1*x1"=>164.17962644641014, "1*x1*x1*x1"=>2103.6762058854374}, "label"=>2.34582297038399}, {"features"=>{"x1"=>20.9519416573912, "1"=>1, "1*x1*x1"=>438.98385921472465, "1*x1*x1*x1"=>9197.564206803343}, "label"=>0.452979606820325}, {"features"=>{"x1"=>34.2002350155681, "1"=>1, "1*x1*x1"=>1169.6560751200902, "1*x1*x1*x1"=>40002.51265649406}, "label"=>3.49824462078296}, {"features"=>{"x1"=>23.944830630756, "1"=>1, "1*x1*x1"=>573.3549139355908, "1*x1*x1*x1"=>13728.886305499407}, "label"=>3.85829855272372}, {"features"=>{"x1"=>17.5232184731391, "1"=>1, "1*x1*x1"=>307.06318565736336, "1*x1*x1*x1"=>5380.735287332051}, "label"=>0.101246545837047}, {"features"=>{"x1"=>19.5963152079201, "1"=>1, "1*x1*x1"=>384.0155697281606, "1*x1*x1*x1"=>7525.290149142054}, "label"=>2.44340629887528}, {"features"=>{"x1"=>20.5996209812623, "1"=>1, "1*x1*x1"=>424.34438457166203, "1*x1*x1*x1"=>8741.333487703248}, "lab

## Question 3.2 (5 Points)

Let's fit this dataset with different polynomial degrees. First, let's see how well linear regression fits the training data. 

Implement a training function that, given a training and testing dataset, trains the model using mini-batch SGD and returns the RMSE error value on both training and testing sets.

In [20]:
def train(sgd, obj, w, train_set, test_set, num_epoch = 100, batch_size = 20)
  

  train_set=train_set["data"]
  test_set=test_set["data"]
  
  num_epoch.times do
    batch_of_examples=train_set.each_slice(batch_size).to_a
    batch_of_examples.each do |batch|
      sgd.update(batch)
    end
  end


  train_rmse= score_regression_model_rmse(train_set, sgd.weights, obj)
  test_rmse=score_regression_model_rmse(test_set, sgd.weights, obj)

  return [train_rmse, test_rmse]
end

:train

In [21]:
def test_32()
  data = read_sparse_data_from_csv "polydata"
  polydata = create_polynomial_features data, 1
  x1 = polydata["data"].collect {|r| r["features"]["x1"]}
  x2 = polydata["data"].collect {|r| r["label"]}
  
  w = Hash.new {|h,k| h[k] = 0.0}
  lr = 1e-3
  obj = LinearRegressionModelL2.new 0.0
  sgd = StochasticGradientDescent.new obj, w, lr

  train_set = polydata
  test_set = polydata
  train_rmse, test_rmse = train(sgd, obj, w, train_set, test_set, num_epoch = 100, batch_size = 20)
  assert_true train_rmse < 2, "1"
  assert_true test_rmse < 2, "2"
  assert_true train_rmse > 0, "3"
  assert_true test_rmse > 0, "4"
  assert_in_delta train_rmse, test_rmse, 1e-5, "5"
end

test_32()

{"classes"=>{}, "features"=>["x1", "1"], "data"=>[{"features"=>{"x1"=>12.8132597900148, "1"=>1}, "label"=>2.34582297038399}, {"features"=>{"x1"=>20.9519416573912, "1"=>1}, "label"=>0.452979606820325}, {"features"=>{"x1"=>34.2002350155681, "1"=>1}, "label"=>3.49824462078296}, {"features"=>{"x1"=>23.944830630756, "1"=>1}, "label"=>3.85829855272372}, {"features"=>{"x1"=>17.5232184731391, "1"=>1}, "label"=>0.101246545837047}, {"features"=>{"x1"=>19.5963152079201, "1"=>1}, "label"=>2.44340629887528}, {"features"=>{"x1"=>20.5996209812623, "1"=>1}, "label"=>3.48748405031445}, {"features"=>{"x1"=>19.734268228103, "1"=>1}, "label"=>-2.52858273819375}, {"features"=>{"x1"=>18.6124126482613, "1"=>1}, "label"=>0.624277395873692}, {"features"=>{"x1"=>30.921573310742, "1"=>1}, "label"=>3.28536421891722}, {"features"=>{"x1"=>26.7268739284682, "1"=>1}, "label"=>0.498349362531655}, {"features"=>{"x1"=>12.4210737567904, "1"=>1}, "label"=>0.49508093700895}, {"features"=>{"x1"=>12.909926115632, "1"=>1}, "l

## Question 3.3 (10 Points)

Implement a simplified version of Gaussian Complexity. Observe that as model complexity increases, test error worsens.

In this simplification, we will compute the average loss of a randomly permuted datasets. Let $H(X,Y)$ be the loss on the training set on a function trained on input examples $x_i\in X$ with labels $y_i\in Y$. Permute the training labels as $y^\prime_i = g y_i$ where $g ~ N(0,1)$ is sampled from a normal distribution with mean 0 and standard deviation 1. Compute the following Gaussian Complexity:

# $R_G(X,H) = -\frac{1}{K} \sum_k H(X,Y^\prime) $

which, in words, is the average of $K$ separate trainings each with a randomly permuted label. We use negative RMSE here to indicate that a more complex model should be more sensitive to permutation and therefore its loss should be lower.

Use ```rng.call()``` to permute the labels.


In [22]:
def gaussian_complexity(dataset, obj)
  rng = Distribution::Normal.rng(0,1, 293891)
  lr = 1e-2
  tr_rmses = []
  te_rmses = []  
  norms = []
  data=dataset["data"]
  test_set=dataset.clone
  
  100.times do |i|
    train_set = dataset.clone

    train_set["data"] = data.collect do |train|
      u = train.clone
      u["label"] *= rng.call()
      u
    end

    w = Hash.new {|h,k| h[k] = 0.0}
    sgd = StochasticGradientDescent.new obj, w, lr
    
    train_rmse, test_rmse =train(sgd, obj, w, train_set, test_set, num_epoch = 100, batch_size = 5)
    norms << norm(w)
    tr_rmses << train_rmse
    te_rmses << test_rmse 

  end  
  result = [mean(tr_rmses), mean(norms), mean(te_rmses)]
  puts result.join("\t")
  result
end

:gaussian_complexity

### Interpretation
In the plot below, you should observe that as the measured complexity increases, so does the test RMSE. Hopefully you see something like this.

![alt text](./expected_complexity_curve.png "Guassian Complexity Sample")


In [ ]:
def test_33()
  stats = Hash.new {|h,k| h[k] = []}
  
  8.times do |i|
    data = read_sparse_data_from_csv "polydata"
    polydata = create_polynomial_features data, i
    obj = LinearRegressionModelL2.new 0.0
    tr_rmse, w_norm, te_rmse = gaussian_complexity(polydata, obj)
    
    stats[:degree] << i
    stats[:train_rmse] << tr_rmse    
    stats[:test_rmse] << te_rmse
    stats[:complexity] << -tr_rmse
  end
  tr_rmse = stats[:train_rmse]
  assert_true(tr_rmse[0] > 0.0)
  assert_true(tr_rmse[0] > tr_rmse[1])
  assert_true(tr_rmse[1] > tr_rmse[2])
  assert_true(tr_rmse[2] > tr_rmse[3])
  assert_true(tr_rmse[2] < 10.0)
  
  te_rmse = stats[:test_rmse]
  assert_true(te_rmse[0] > 0.0)
  assert_true(te_rmse[0] < te_rmse[1])
  assert_true(te_rmse[1] < te_rmse[2])
  assert_true(te_rmse.last < 10.0)
  
  z_plot = Nyaplot::Plot.new
  z_plot.x_label("Model Complexity").y_label("Test RMSE")
  z_plot.add(:line, stats[:complexity], stats[:test_rmse]).color(:black)
  z_plot.show()  
end
test_33()

{"classes"=>{}, "features"=>["x1", "1"], "data"=>[{"features"=>{"x1"=>12.8132597900148, "1"=>1}, "label"=>2.34582297038399}, {"features"=>{"x1"=>20.9519416573912, "1"=>1}, "label"=>0.452979606820325}, {"features"=>{"x1"=>34.2002350155681, "1"=>1}, "label"=>3.49824462078296}, {"features"=>{"x1"=>23.944830630756, "1"=>1}, "label"=>3.85829855272372}, {"features"=>{"x1"=>17.5232184731391, "1"=>1}, "label"=>0.101246545837047}, {"features"=>{"x1"=>19.5963152079201, "1"=>1}, "label"=>2.44340629887528}, {"features"=>{"x1"=>20.5996209812623, "1"=>1}, "label"=>3.48748405031445}, {"features"=>{"x1"=>19.734268228103, "1"=>1}, "label"=>-2.52858273819375}, {"features"=>{"x1"=>18.6124126482613, "1"=>1}, "label"=>0.624277395873692}, {"features"=>{"x1"=>30.921573310742, "1"=>1}, "label"=>3.28536421891722}, {"features"=>{"x1"=>26.7268739284682, "1"=>1}, "label"=>0.498349362531655}, {"features"=>{"x1"=>12.4210737567904, "1"=>1}, "label"=>0.49508093700895}, {"features"=>{"x1"=>12.909926115632, "1"=>1}, "l

2.3821877511017044	0.22453577245389916	2.4749250444604027
{"classes"=>{}, "features"=>["x1", "1"], "data"=>[{"features"=>{"x1"=>12.8132597900148, "1"=>1, "1*x1*x1"=>164.17962644641014, "1*x1*x1*x1"=>2103.6762058854374, "1*x1*x1*x1*x1"=>26954.94974008277}, "label"=>2.34582297038399}, {"features"=>{"x1"=>20.9519416573912, "1"=>1, "1*x1*x1"=>438.98385921472465, "1*x1*x1*x1"=>9197.564206803343, "1*x1*x1*x1*x1"=>192706.8286510532}, "label"=>0.452979606820325}, {"features"=>{"x1"=>34.2002350155681, "1"=>1, "1*x1*x1"=>1169.6560751200902, "1*x1*x1*x1"=>40002.51265649406, "1*x1*x1*x1*x1"=>1368095.3340653342}, "label"=>3.49824462078296}, {"features"=>{"x1"=>23.944830630756, "1"=>1, "1*x1*x1"=>573.3549139355908, "1*x1*x1*x1"=>13728.886305499407, "1*x1*x1*x1*x1"=>328735.8573340888}, "label"=>3.85829855272372}, {"features"=>{"x1"=>17.5232184731391, "1"=>1, "1*x1*x1"=>307.06318565736336, "1*x1*x1*x1"=>5380.735287332051, "1*x1*x1*x1*x1"=>94287.79998604841}, "label"=>0.101246545837047}, {"features"=>{

2.336661448738838	0.2686236333006467	2.5189091478081465
{"classes"=>{}, "features"=>["x1", "1"], "data"=>[{"features"=>{"x1"=>12.8132597900148, "1"=>1, "1*x1*x1"=>164.17962644641014, "1*x1*x1*x1"=>2103.6762058854374, "1*x1*x1*x1*x1"=>26954.94974008277, "1*x1*x1*x1*x1*x1"=>345380.7736464725, "1*x1*x1*x1*x1*x1*x1"=>4425453.579208549, "1*x1*x1*x1*x1*x1*x1*x1"=>56704486.39904998}, "label"=>2.34582297038399}, {"features"=>{"x1"=>20.9519416573912, "1"=>1, "1*x1*x1"=>438.98385921472465, "1*x1*x1*x1"=>9197.564206803343, "1*x1*x1*x1*x1"=>192706.8286510532, "1*x1*x1*x1*x1*x1"=>4037582.230877749, "1*x1*x1*x1*x1*x1*x1"=>84595187.33827001, "1*x1*x1*x1*x1*x1*x1*x1"=>1772433429.607512}, "label"=>0.452979606820325}, {"features"=>{"x1"=>34.2002350155681, "1"=>1, "1*x1*x1"=>1169.6560751200902, "1*x1*x1*x1"=>40002.51265649406, "1*x1*x1*x1*x1"=>1368095.3340653342, "1*x1*x1*x1*x1*x1"=>46789181.94873658, "1*x1*x1*x1*x1*x1*x1"=>1600201018.8329675, "1*x1*x1*x1*x1*x1*x1*x1"=>54727250916.239}, "label"=>3.498244

Test::Unit::AssertionFailedError: <true> expected but was
<false>

## Question 3.4 (5 points)

Does regularization reduce the Gaussian Complexity? Copy ```test_33``` above and modify it to select a fixed value for the polynomial degree, say $k=5$. Validate that both norm and complexity decreases as you increase the regularization parameter. Due to limitations in SGD, some large regularization values may cause the trainer to diverge. Try adjusting the learning rate. 


### Interpretation

We should expect to see that as we decrease the regularization penalty $\lambda$ the learned norm of the weight increases and therefore complexity of the model increases. Since we established above that the test RMSE increases as complexity increases, we can conclude that lower regularization penalty leads to higher complexity and worse test performance. Your plot should look like this:

![Expected complexity vs. weight norm](./expected_compexity_weight_norm.png)


In [24]:
def complexity_vs_norm()
  stats = Hash.new {|h,k| h[k] = []}
  data = read_sparse_data_from_csv "polydata"

  [0.0, 0.1, 0.5, 1.0, 1.5, 2.0, 5.0, 10.0, 15.0, 100.0].each do |reg|

    polydata = create_polynomial_features data, 5
    obj = LinearRegressionModelL2.new reg
    tr_rmse, w_norm, te_rmse = gaussian_complexity(polydata, obj)
    
    stats[:regularizer] << reg
    stats[:train_rmse] << tr_rmse    
    stats[:test_rmse] << te_rmse
    stats[:norms] << w_norm    
    stats[:complexity] << -tr_rmse
  end
  
  return stats
end

:complexity_vs_norm

In [25]:
def test_34()
  stats = complexity_vs_norm()
  
  assert_true(stats[:train_rmse].all? {|t| t > 0 and t < 5})
  assert_true(stats[:test_rmse].all? {|t| t > 0 and t < 5})  
  assert_true(stats[:norms].all? {|t| t > 0 and t < 10})    
  z_plot = Nyaplot::Plot.new
  z_plot.x_label("Weight Norm").y_label("Model Complexity")
  z_plot.add(:line, stats[:norms], stats[:complexity]).color(:black)
  z_plot.show()  
end

test_34()

{"classes"=>{}, "features"=>["x1", "1"], "data"=>[{"features"=>{"x1"=>12.8132597900148, "1"=>1, "1*x1*x1"=>164.17962644641014, "1*x1*x1*x1"=>2103.6762058854374, "1*x1*x1*x1*x1"=>26954.94974008277, "1*x1*x1*x1*x1*x1"=>345380.7736464725}, "label"=>2.34582297038399}, {"features"=>{"x1"=>20.9519416573912, "1"=>1, "1*x1*x1"=>438.98385921472465, "1*x1*x1*x1"=>9197.564206803343, "1*x1*x1*x1*x1"=>192706.8286510532, "1*x1*x1*x1*x1*x1"=>4037582.230877749}, "label"=>0.452979606820325}, {"features"=>{"x1"=>34.2002350155681, "1"=>1, "1*x1*x1"=>1169.6560751200902, "1*x1*x1*x1"=>40002.51265649406, "1*x1*x1*x1*x1"=>1368095.3340653342, "1*x1*x1*x1*x1*x1"=>46789181.94873658}, "label"=>3.49824462078296}, {"features"=>{"x1"=>23.944830630756, "1"=>1, "1*x1*x1"=>573.3549139355908, "1*x1*x1*x1"=>13728.886305499407, "1*x1*x1*x1*x1"=>328735.8573340888, "1*x1*x1*x1*x1*x1"=>7871524.426121125}, "label"=>3.85829855272372}, {"features"=>{"x1"=>17.5232184731391, "1"=>1, "1*x1*x1"=>307.06318565736336, "1*x1*x1*x1"=>5

2.349486730738937	0.25652081844966423	2.504126214269124
{"classes"=>{}, "features"=>["x1", "1"], "data"=>[{"features"=>{"x1"=>12.8132597900148, "1"=>1, "1*x1*x1"=>164.17962644641014, "1*x1*x1*x1"=>2103.6762058854374, "1*x1*x1*x1*x1"=>26954.94974008277, "1*x1*x1*x1*x1*x1"=>345380.7736464725}, "label"=>2.34582297038399}, {"features"=>{"x1"=>20.9519416573912, "1"=>1, "1*x1*x1"=>438.98385921472465, "1*x1*x1*x1"=>9197.564206803343, "1*x1*x1*x1*x1"=>192706.8286510532, "1*x1*x1*x1*x1*x1"=>4037582.230877749}, "label"=>0.452979606820325}, {"features"=>{"x1"=>34.2002350155681, "1"=>1, "1*x1*x1"=>1169.6560751200902, "1*x1*x1*x1"=>40002.51265649406, "1*x1*x1*x1*x1"=>1368095.3340653342, "1*x1*x1*x1*x1*x1"=>46789181.94873658}, "label"=>3.49824462078296}, {"features"=>{"x1"=>23.944830630756, "1"=>1, "1*x1*x1"=>573.3549139355908, "1*x1*x1*x1"=>13728.886305499407, "1*x1*x1*x1*x1"=>328735.8573340888, "1*x1*x1*x1*x1*x1"=>7871524.426121125}, "label"=>3.85829855272372}, {"features"=>{"x1"=>17.5232184731391

2.356343219352321	0.22491987682976738	2.4919806151554202
{"classes"=>{}, "features"=>["x1", "1"], "data"=>[{"features"=>{"x1"=>12.8132597900148, "1"=>1, "1*x1*x1"=>164.17962644641014, "1*x1*x1*x1"=>2103.6762058854374, "1*x1*x1*x1*x1"=>26954.94974008277, "1*x1*x1*x1*x1*x1"=>345380.7736464725}, "label"=>2.34582297038399}, {"features"=>{"x1"=>20.9519416573912, "1"=>1, "1*x1*x1"=>438.98385921472465, "1*x1*x1*x1"=>9197.564206803343, "1*x1*x1*x1*x1"=>192706.8286510532, "1*x1*x1*x1*x1*x1"=>4037582.230877749}, "label"=>0.452979606820325}, {"features"=>{"x1"=>34.2002350155681, "1"=>1, "1*x1*x1"=>1169.6560751200902, "1*x1*x1*x1"=>40002.51265649406, "1*x1*x1*x1*x1"=>1368095.3340653342, "1*x1*x1*x1*x1*x1"=>46789181.94873658}, "label"=>3.49824462078296}, {"features"=>{"x1"=>23.944830630756, "1"=>1, "1*x1*x1"=>573.3549139355908, "1*x1*x1*x1"=>13728.886305499407, "1*x1*x1*x1*x1"=>328735.8573340888, "1*x1*x1*x1*x1*x1"=>7871524.426121125}, "label"=>3.85829855272372}, {"features"=>{"x1"=>17.523218473139

2.3641114823604057	0.1966467774506019	2.4823460384883003
{"classes"=>{}, "features"=>["x1", "1"], "data"=>[{"features"=>{"x1"=>12.8132597900148, "1"=>1, "1*x1*x1"=>164.17962644641014, "1*x1*x1*x1"=>2103.6762058854374, "1*x1*x1*x1*x1"=>26954.94974008277, "1*x1*x1*x1*x1*x1"=>345380.7736464725}, "label"=>2.34582297038399}, {"features"=>{"x1"=>20.9519416573912, "1"=>1, "1*x1*x1"=>438.98385921472465, "1*x1*x1*x1"=>9197.564206803343, "1*x1*x1*x1*x1"=>192706.8286510532, "1*x1*x1*x1*x1*x1"=>4037582.230877749}, "label"=>0.452979606820325}, {"features"=>{"x1"=>34.2002350155681, "1"=>1, "1*x1*x1"=>1169.6560751200902, "1*x1*x1*x1"=>40002.51265649406, "1*x1*x1*x1*x1"=>1368095.3340653342, "1*x1*x1*x1*x1*x1"=>46789181.94873658}, "label"=>3.49824462078296}, {"features"=>{"x1"=>23.944830630756, "1"=>1, "1*x1*x1"=>573.3549139355908, "1*x1*x1*x1"=>13728.886305499407, "1*x1*x1*x1*x1"=>328735.8573340888, "1*x1*x1*x1*x1*x1"=>7871524.426121125}, "label"=>3.85829855272372}, {"features"=>{"x1"=>17.523218473139

2.4055896739390596	0.09480726140392648	2.457134708730127
{"classes"=>{}, "features"=>["x1", "1"], "data"=>[{"features"=>{"x1"=>12.8132597900148, "1"=>1, "1*x1*x1"=>164.17962644641014, "1*x1*x1*x1"=>2103.6762058854374, "1*x1*x1*x1*x1"=>26954.94974008277, "1*x1*x1*x1*x1*x1"=>345380.7736464725}, "label"=>2.34582297038399}, {"features"=>{"x1"=>20.9519416573912, "1"=>1, "1*x1*x1"=>438.98385921472465, "1*x1*x1*x1"=>9197.564206803343, "1*x1*x1*x1*x1"=>192706.8286510532, "1*x1*x1*x1*x1*x1"=>4037582.230877749}, "label"=>0.452979606820325}, {"features"=>{"x1"=>34.2002350155681, "1"=>1, "1*x1*x1"=>1169.6560751200902, "1*x1*x1*x1"=>40002.51265649406, "1*x1*x1*x1*x1"=>1368095.3340653342, "1*x1*x1*x1*x1*x1"=>46789181.94873658}, "label"=>3.49824462078296}, {"features"=>{"x1"=>23.944830630756, "1"=>1, "1*x1*x1"=>573.3549139355908, "1*x1*x1*x1"=>13728.886305499407, "1*x1*x1*x1*x1"=>328735.8573340888, "1*x1*x1*x1*x1*x1"=>7871524.426121125}, "label"=>3.85829855272372}, {"features"=>{"x1"=>17.523218473139

2.4549073931729506	0.023232289331325928	2.4398788098354207


#<Nyaplot::Frame:0x00000000029d9070 @properties={:panes=>[#<Nyaplot::Plot:0x00000000029da970 @properties={:diagrams=>[#<Nyaplot::Diagram:0x00000000029d95c0 @properties={:type=>:line, :options=>{:x=>"data0", :y=>"data1", :color=>:black}, :data=>"d3d46ed7-0e6f-40c0-9447-7aeb5228e5d0"}, @xrange=[0.023232289331325928, 0.26046185702227864], @yrange=[-2.4549073931729506, -2.3487658990284013]>], :options=>{:x_label=>"Weight Norm", :y_label=>"Model Complexity", :zoom=>true, :width=>700, :xrange=>[0.023232289331325928, 0.26046185702227864], :yrange=>[-2.4549073931729506, -2.3487658990284013]}}>], :data=>{"d3d46ed7-0e6f-40c0-9447-7aeb5228e5d0"=>#<Nyaplot::DataFrame:0x00000000029da380 @name="d3d46ed7-0e6f-40c0-9447-7aeb5228e5d0", @rows=[{:data0=>0.26046185702227864, :data1=>-2.3487658990284013}, {:data0=>0.25652081844966423, :data1=>-2.349486730738937}, {:data0=>0.24165516949603938, :data1=>-2.3524733048132274}, {:data0=>0.22491987682976738, :data1=>-2.356343219352321}, {:data0=>0.2099928125341367, :data1=>-2.360251032657988}, {:data0=>0.1966467774506019, :data1=>-2.3641114823604057}, {:data0=>0.14043236778271587, :data1=>-2.3842614524715575}, {:data0=>0.09480726140392648, :data1=>-2.4055896739390596}, {:data0=>0.07296366992180806, :data1=>-2.4176792208734112}, {:data0=>0.023232289331325928, :data1=>-2.4549073931729506}]>}, :extension=>[]}>

## Question 4.1 (10 Points)

Moving on to classification, implement L2 regularization for Logisitic Regression. This should follow closely what you did in Question 2.X above. 

Use the Log Loss formulation, $\log(1 + \exp(-y\cdot \hat{y}))$ when calculating the objective value.

Map all labels to either 1 or -1.

In [26]:
class LogisticRegressionModelL2
  def initialize reg_param
    @reg_param = reg_param
  end

  def predict row, w
    x=row["features"]
    return 1.0 / (1.0 + Math.exp(-dot(x,w )))
  end
  
  def adjust w
    w.each_key {|k| w[k] = 0.0 if w[k].nan? or w[k].infinite?}
    w.each_key {|k| w[k] = 0.0 if w[k].abs > 1e5 }
  end

  def func data, w
   
    n = data.size
    sum = 0
    n.times do |i|
      example = data[i]
      sum += ((1.0+example["label"]) * Math.log(predict(example,w)) + (1.0-example["label"]) * Math.log(1.0-predict(example,w)))/2
    end
    penalty = 0.5*(@reg_param)*(norm(w))**2
    return (penalty + (-1.0*sum/n.to_f))
    
  end

  
end

:func

In [27]:
### TEST ###
def test_41_0()
  #Test loss without regularization
  m = LogisticRegressionModelL2.new 0.0
  w = {"x1" => 1.0, "x2" => -3.0}
  
  examples = [
      {"features" => {"x1" => 0.7, "x2" => -0.3}, "label" => 1.0},
      {"features" => {"x1" => -2.7, "x2" => -1.3}, "label" => -1.0}  
  ]
  
  dataset = {"data" => examples}
  e1 = 0.1839007409
  assert_in_delta e1, m.func(examples[0,1], w), 1e-2, "1"
  
  e2 = 1.4632824673
  assert_in_delta e2, m.func(examples[1,1], w), 1e-2, "2"    

  #Loss for 2 examples should be their average
  assert_in_delta (e1 + e2) / 2.0, m.func(examples, w), 1e-2, "3"  
end

test_41_0()

In [28]:
### TEST ###
def test_41_1()
  # Test loss with regularization
  m = LogisticRegressionModelL2.new 1.7
  w = {"x1" => 1.0, "x2" => -3.0}
  
  examples = [
      {"features" => {"x1" => 0.7, "x2" => -0.3}, "label" => 1.0},
      {"features" => {"x1" => -2.7, "x2" => -1.3}, "label" => -1.0}  
  ]
  
  dataset = {"data" => examples}

  assert_in_delta 9.3235916041, m.func(examples, w), 1e-2, "1"
end

test_41_1()

## Question 4.2 (5 Points)

Implement the gradient for L2 regularized Logisitic Regression. As in Assignment 5, use the 0 / 1 version of the loss to simplify the derivation.

In [29]:
class LogisticRegressionModelL2
  def grad data, w
    
    gradient = Hash.new {|h,k| h[k] = 0.0}
    n = data.size   
           
    n.times do |i|
      example = data[i]
      x = example["features"]
      p = predict(example, w)
      xSize = x.size
      xSize.times do |j|
        xKey = x.keys[j]
        a = -1*(1-p)*x[xKey]
        b = p*x[xKey]
        gradient[xKey] = gradient[xKey] + (a*(1+example["label"])+b*(1-example["label"]))/2
      end     
    end    
   
    gSize = gradient.size
   
    gSize.times do |i|
      gKey = gradient.keys[i]
      gradient[gKey] = ((gradient[gKey]).to_f)/n.to_f
      gradient[gKey] += ((@reg_param)*w[gKey])
    end
   
    return gradient
  end
end

:grad

In [30]:
class LinearRegressionModelL2
  def grad examples, w
    g = Hash.new {|h,k| h[k] = 0.0}
    n = examples.size().to_f
    
    examples.each do |ex|
        features = ex["features"]
        prediction = predict(ex, w)
        label = ex["label"]
        diff = prediction - label
        features.each do |f_key, f_val|
          g[f_key] = 0.0 if g[f_key].nan?
          g[f_key] += f_val * diff
        end
    end
    
    w.each do |w_key, w_val|
        second_term = n > 0.0 ? g[w_key] / n : g[w_key]
        g[w_key] = @reg_param * w_val + second_term
    end
    
    
    return g
  end
end

:grad

In [31]:
### TEST ###
def test_42_0()
  #First without regularization
  m = LogisticRegressionModelL2.new 0.0
  w = {"x1" => 1.0, "x2" => -3.0}
  examples = [
      {"features" => {"x1" => 0.7, "x2" => -0.3}, "label" => 1.0},
      {"features" => {"x1" => -2.7, "x2" => -1.3}, "label" => -1.0}  
  ]
  dataset = {"data" => examples}
  
  g1_1 = -0.1175871304
  assert_in_delta g1_1, m.grad(examples[0,1], w)["x1"], 1e-2, "1"
  
  g2_1 =  -2.0750169154
  assert_in_delta g2_1, m.grad(examples[1,1], w)["x1"], 1e-2, "2"    
  assert_in_delta (g1_1 + g2_1) / 2.0, m.grad(examples, w)["x1"], 1e-2, "3"  
end

test_42_0()

In [32]:
### TEST ###
def test_42_1()
  m = LogisticRegressionModelL2.new 1.7
  w = {"x1" => 1.0, "x2" => -3.0}
  examples = [
      {"features" => {"x1" => 0.7, "x2" => -0.3}, "label" => 1.0},
      {"features" => {"x1" => -2.7, "x2" => -1.3}, "label" => -1.0}  
  ]
  dataset = {"data" => examples}
  
  g1_1 = -0.1175871304
  g2_1 =  -2.0750169154
  assert_in_delta 1.0 * 1.7 + (g1_1 + g2_1) / 2.0, m.grad(examples, w)["x1"], 1e-2, "5"
end

test_42_1()

## Question 4.3 (2 points)

Implement a function that will score your logistic regression model and return an array of pairs of (score, class label).

For the example in the test below, your output should look like this:

```ruby
[
    [0.8320183851339245, 1.0], 
    [0.7685247834990178, 0.0]
]
```

In [31]:
def score_binary_classification_model(data, weights, model)
  sum=0
  
  result=[]

  data.each do |key, array|

    result+=[[model.predict(key, weights),key["label"]]]
  end
  
  return result
  
end

:score_binary_classification_model

In [32]:
### TEST ###
def test_43()
  m = LogisticRegressionModelL2.new 888.0
  w = {"x1" => 1.0, "x2" => -3.0}
  examples = [
      {"features" => {"x1" => 0.7, "x2" => -0.3}, "label" => 1.0},
      {"features" => {"x1" => -2.7, "x2" => -1.3}, "label" => 0.0}  
  ]
  
  e1 = 0.8320183851
  e2 = 0.7685247835
  
  scores = score_binary_classification_model(examples, w, m)
  puts scores
  
  assert_in_delta e1, scores[0][0], 1e-2, "1"
  assert_in_delta e2, scores[1][0], 1e-2, "2"  
  assert_in_delta 1.0, scores[0][1], 1e-2, "3"
  assert_in_delta 0.0, scores[1][1], 1e-2, "4"  
end

test_43()

[[0.8320183851339245, 1.0], [0.7685247834990178, 0.0]]


## Question 4.4 (10 Points)

Given the scores defined above, recall that accuracy, true positive, false positive, precision, and recall are defined only for a single threshold $t$. Define each of these measures, given a cut point and assume here that the cut point is defined as $s \ge t$ for score $s$ and threshold $t$. Each function should apply only to the positive class, i.e., when ```label > 0```.

In [33]:
def accuracy(scores, t)
  
  false_positive=0
  false_negative=0
  true_positive=0
  true_negative=0
  total=0
  scores.each do |item|
    if item[0]<t and item[1]!=1.0
      true_negative+=1.0
    elsif item[0]>=t and item[1]==1.0
      true_positive+=1.0
    elsif item[0]>=t and item[1]!=1.0
      false_positive+=1.0
    elsif item[0]<t and item[1]==1.0
      false_negative+=1.0
    end 
  end

  return (true_positive+true_negative)/scores.size
end

:accuracy

In [34]:
### TEST ###
def test_44_scores()
  [
    [0.22, 0.0],
    [0.23, 0.0], 
    [0.24, 1.0], 
    [0.25, 0.0],     
    [0.49, 0.0],
    [0.5, 1.0],
    [0.73, 0.0],    
    [0.74, 0.0],
    [0.75, 1.0],    
    [0.76, 1.0],
  ]
end

def test_44_0()
  scores = test_44_scores()
  assert_in_delta 0.4, accuracy(scores, 0.0), 1e-3, "t = 0.0"
  assert_in_delta 0.5, accuracy(scores, 0.25), 1e-3, "t = 0.25"
  assert_in_delta 0.7, accuracy(scores, 0.5), 1e-3, "t = 0.5"
  assert_in_delta 0.8, accuracy(scores, 0.75), 1e-3, "t = 0.75"
  assert_in_delta 0.6, accuracy(scores, 1.0), 1e-3, "t = 1.0"
end

test_44_0()

In [35]:
def true_positive(scores, t)
  false_positive=0
  false_negative=0
  true_positive=0
  true_negative=0
  total=0
  scores.each do |item|
    if item[0]<t and item[1]!=1.0
      true_negative+=1.0
    elsif item[0]>=t and item[1]==1.0
      true_positive+=1.0
    elsif item[0]>=t and item[1]!=1.0
      false_positive+=1.0
    elsif item[0]<t and item[1]==1.0
      false_negative+=1.0
    end 
  end
      
  return (true_positive)/(true_positive+false_negative)
end

:true_positive

In [36]:
### TEST ###
def test_44_1()
  scores = test_44_scores()
  assert_in_delta 1.0, true_positive(scores, 0.0), 1e-3, "t = 0.0"
  assert_in_delta 0.75, true_positive(scores, 0.25), 1e-3, "t = 0.25"
  assert_in_delta 0.75, true_positive(scores, 0.5), 1e-3, "t = 0.5"
  assert_in_delta 0.5, true_positive(scores, 0.75), 1e-3, "t = 0.75"
  assert_in_delta 0.0, true_positive(scores, 1.0), 1e-3, "t = 1.0"
end

test_44_1()

In [37]:
def false_positive(scores, t)

  false_positive=0
  false_negative=0
  true_positive=0
  true_negative=0
  total=0
  scores.each do |item|
    if item[0]<t and item[1]!=1.0
      true_negative+=1.0
    elsif item[0]>=t and item[1]==1.0
      true_positive+=1.0
    elsif item[0]>=t and item[1]!=1.0
      false_positive+=1.0
    elsif item[0]<t and item[1]==1.0
      false_negative+=1.0
    end 
  end
  
  return (false_positive)/((true_negative+false_positive))
end

:false_positive

In [38]:
### TEST ###
def test_44_2()
  scores = test_44_scores()
  assert_in_delta 1.0, false_positive(scores, 0.0), 1e-3, "t = 0.0"
  assert_in_delta 0.6666, false_positive(scores, 0.25), 1e-3, "t = 0.25"
  assert_in_delta 0.3333, false_positive(scores, 0.5), 1e-3, "t = 0.5"
  assert_in_delta 0.0, false_positive(scores, 0.75), 1e-3, "t = 0.75"
  assert_in_delta 0.0, false_positive(scores, 1.0), 1e-3, "t = 1.0"
end

test_44_2()

In [39]:
# For precision, set the precision to 1.0 if there are no scored examples
def precision(scores, t)

  false_positive=0
  false_negative=0
  true_positive=0
  true_negative=0
  total=0
  scores.each do |item|
    if item[0]<t and item[1]!=1.0
      true_negative+=1.0
    elsif item[0]>=t and item[1]==1.0
      true_positive+=1.0
    elsif item[0]>=t and item[1]!=1.0
      false_positive+=1.0
    elsif item[0]<t and item[1]==1.0
      false_negative+=1.0
    end 
  end
  
  if (true_positive+false_positive)==0
    return 1
  end
  return (true_positive)/((true_positive+false_positive))
end

:precision

In [40]:
### TEST ###
def test_44_3()
  scores = test_44_scores()
  assert_in_delta 0.4, precision(scores, 0.0), 1e-3, "t = 0.0"
  assert_in_delta 0.42857, precision(scores, 0.25), 1e-3, "t = 0.25"
  assert_in_delta 0.6, precision(scores, 0.5), 1e-3, "t = 0.5"
  assert_in_delta 1.0, precision(scores, 0.75), 1e-3, "t = 0.75"
  assert_in_delta 1.0, precision(scores, 1.0), 1e-3, "t = 1.0"
end

test_44_3()

In [41]:
def recall(scores, t)
  false_positive=0
  false_negative=0
  true_positive=0
  true_negative=0
  total=0
  scores.each do |item|
    if item[0]<t and item[1]!=1.0
      true_negative+=1.0
    elsif item[0]>=t and item[1]==1.0
      true_positive+=1.0
    elsif item[0]>=t and item[1]!=1.0
      false_positive+=1.0
    elsif item[0]<t and item[1]==1.0
      false_negative+=1.0
    end 
  end

  return (true_positive)/(true_positive+false_negative)
end

:recall

In [42]:
### TEST ###
def test_44_4()
  scores = test_44_scores()
  assert_in_delta 1.0, recall(scores, 0.0), 1e-3, "t = 0.0"
  assert_in_delta 0.75, recall(scores, 0.25), 1e-3, "t = 0.25"
  assert_in_delta 0.75, recall(scores, 0.5), 1e-3, "t = 0.5"
  assert_in_delta 0.5, recall(scores, 0.75), 1e-3, "t = 0.75"
  assert_in_delta 0.0, recall(scores, 1.0), 1e-3, "t = 1.0"
end

test_44_4()

## Question 4.5 (5 points)

In cross validation, we simulate having real test dataset but taking several samples of the data. Cross validation proceeds in $k$ folds deterministically where for 100 examples and 5 folds, we have in the first round, the first 20 examples in test set and the rest in training. For the second rounds, we have examples 20-39 for testing and the remainder for training.

Implement this with a callback ```block``` as follows:

```ruby
def cross_validate dataset, folds, &block
  examples = data["data"]

  folds.times do |fold|
    train_data = dataset.clone
    train_data["data"] = ##CV training examples
    
    test_data = data.clone
    test_data["data"] = ##CV testing examples
    
    ## Call the callback like this:
    yield train_data, test_data, fold
  end
end
```

**Note**: Do not shuffle the data. Preserve the order. Also, make a copy of the dataset so as not to overwrite any datasets or examples. 

In [43]:
def cross_validate dataset, folds, &block
  examples = dataset["data"]
  slice=(dataset["data"].size)/folds

  folds.times do |fold|

    train_data = dataset.clone
    train_data["data"] = train_data["data"]-train_data["data"][fold*slice..((fold+1)*slice-1)]

    test_data = dataset.clone
    test_data["data"] = test_data["data"][fold*slice..((fold+1)*slice-1)]

    ## Call the callback like this:
    yield train_data, test_data, fold
  end
end

:cross_validate

In [44]:
def test_45()
  examples = [
    {"features" => {"x1" => 1}},
    {"features" => {"x2" => 1}},
    {"features" => {"x3" => 1}},
    {"features" => {"x4" => 1}},    
    {"features" => {"x5" => 1}},    
  ]
  dataset = {"data" => examples}
  
  train_sets = []
  test_sets = []
  folds = []
  
  cross_validate dataset, 5 do |train_data, test_data, fold|
    puts "Fold: #{fold}:", "Train Data", train_data, "Test Data:", test_data
    train_sets << train_data
    test_sets << test_data
    folds << fold
    puts ""
  end

  assert_equal [0,1,2,3,4], folds
  assert_true(test_sets[0]["data"][0]["features"].has_key? "x1")
  assert_equal 4, train_sets[0]["data"].size

  assert_true(test_sets[3]["data"][0]["features"].has_key? "x4")
  assert_equal 4, train_sets[3]["data"].size
  assert_true(train_sets[3]["data"][0]["features"].has_key? "x1")

  
end
test_45()

Fold: 0:
Train Data
{"data"=>[{"features"=>{"x2"=>1}}, {"features"=>{"x3"=>1}}, {"features"=>{"x4"=>1}}, {"features"=>{"x5"=>1}}]}
Test Data:
{"data"=>[{"features"=>{"x1"=>1}}]}

Fold: 1:
Train Data
{"data"=>[{"features"=>{"x1"=>1}}, {"features"=>{"x3"=>1}}, {"features"=>{"x4"=>1}}, {"features"=>{"x5"=>1}}]}
Test Data:
{"data"=>[{"features"=>{"x2"=>1}}]}

Fold: 2:
Train Data
{"data"=>[{"features"=>{"x1"=>1}}, {"features"=>{"x2"=>1}}, {"features"=>{"x4"=>1}}, {"features"=>{"x5"=>1}}]}
Test Data:
{"data"=>[{"features"=>{"x3"=>1}}]}

Fold: 3:
Train Data
{"data"=>[{"features"=>{"x1"=>1}}, {"features"=>{"x2"=>1}}, {"features"=>{"x3"=>1}}, {"features"=>{"x5"=>1}}]}
Test Data:
{"data"=>[{"features"=>{"x4"=>1}}]}

Fold: 4:
Train Data
{"data"=>[{"features"=>{"x1"=>1}}, {"features"=>{"x2"=>1}}, {"features"=>{"x3"=>1}}, {"features"=>{"x4"=>1}}]}
Test Data:
{"data"=>[{"features"=>{"x5"=>1}}]}



## Question 5.1 (5 Points)

Given an array of pairs of score and class label (0,1), calculate the AUC metric. It is not necessary to draw the curve, but you are welcome to do that. Assume scores are not sorted.

Recall the definition of AUC as either the area under the ROC curve or the probability of mis-ranking a positive example. Choose one of these methods for the implementation. 

In [46]:
def calc_auc_only(scores)
  
  puts "scores"
  puts scores
    
  arrayS = scores.sort_by {|(v,k)| -v}

    
  puts "arrayS"
  puts arrayS
  
  fp=[]
  tp=[]
  totalN=0
  totalP=0
  arrayS.each do |item|
    if arrayS[1]<1
      totalN+=1
    else
      totalP+=1
    end
  end
  
  puts "total positive"
  puts totalP
  
  puts "total negative"
  puts totalN
  
  arrayS.each do |item|
    if arrayS[1]<1
      totalN+=1
    else
      totalP+=1
    end
  end
  
  
  max = scores[0][0]
  min = scores[0][0]
 
  (scores.size).times do |i|
    if(max < scores[i][0])
      max = scores[i][0]
    end
   
    if(min > scores[i][0])
      min = scores[i][0]
    end
  end
 
  cutCount = 100
  auc = 0.0
  prevTPositive = 1.0
  prevFPositive = 1.0
  t = min
  tIncr = (max-min)/cutCount.to_f
 
 
  cutCount.times do |i|
    t += tIncr
    auc += (-0.5)*(true_positive(scores, t) + prevTPositive)*(false_positive(scores, t) - prevFPositive)
    prevTPositive = true_positive(scores, t)
    prevFPositive = false_positive(scores, t)
  end
 
  return auc
end
 


:calc_auc_only

In [47]:

def calc_auc_only(scores)
  max = scores[0][0]
  min = scores[0][0]
 
  (scores.size).times do |i|
    if(max < scores[i][0])
      max = scores[i][0]
    end
   
    if(min > scores[i][0])
      min = scores[i][0]
    end
  end
 
  cutCount = 100
  auc = 0.0
  prevTPositive = 1.0
  prevFPositive = 1.0
  t = min
  tIncr = (max-min)/cutCount.to_f
 
 
  cutCount.times do |i|
    t += tIncr
    auc += (-0.5)*(true_positive(scores, t) + prevTPositive)*(false_positive(scores, t) - prevFPositive)
    prevTPositive = true_positive(scores, t)
    prevFPositive = false_positive(scores, t)
  end
 
  return auc
end
 


:calc_auc_only

In [47]:
def test_51()
  good_model = [[0.9, 1], [0.89, 1], [0.7, 0], [0.8, 1], [0.8, 0], [0.7, 1], [0.6, 0], [0.5, 0], [0.1, 0]]
  assert_true(calc_auc_only(good_model) > 0.8)
  assert_true(calc_auc_only(good_model) < 1)
  
  srand(777)
  ok_model = Array.new(100) {|i| [100 - i, (rand < (100 - i) / 100.0) ? 1 : 0] }
  ok_auc = calc_auc_only(ok_model)
  assert_in_delta(0.8631239935587761, ok_auc, 1e-3)
  
  bad_model = Array.new(1000) {|i| [1000 - i, rand < 0.5 ? 1 : 0] }
  bad_auc = calc_auc_only(bad_model)
  assert_in_delta(0.5, bad_auc, 5e-2)

end

test_51()

scores
[[0.9, 1], [0.89, 1], [0.7, 0], [0.8, 1], [0.8, 0], [0.7, 1], [0.6, 0], [0.5, 0], [0.1, 0]]
arrayS
[[0.9, 1], [0.89, 1], [0.8, 1], [0.8, 0], [0.7, 0], [0.7, 1], [0.6, 0], [0.5, 0], [0.1, 0]]
scores
[[0.9, 1], [0.89, 1], [0.7, 0], [0.8, 1], [0.8, 0], [0.7, 1], [0.6, 0], [0.5, 0], [0.1, 0]]
arrayS
[[0.9, 1], [0.89, 1], [0.8, 1], [0.8, 0], [0.7, 0], [0.7, 1], [0.6, 0], [0.5, 0], [0.1, 0]]
scores
[[100, 1], [99, 1], [98, 1], [97, 1], [96, 1], [95, 1], [94, 1], [93, 1], [92, 1], [91, 1], [90, 1], [89, 1], [88, 1], [87, 1], [86, 0], [85, 1], [84, 1], [83, 1], [82, 1], [81, 1], [80, 1], [79, 1], [78, 1], [77, 1], [76, 1], [75, 0], [74, 1], [73, 1], [72, 1], [71, 0], [70, 1], [69, 1], [68, 1], [67, 1], [66, 1], [65, 1], [64, 1], [63, 0], [62, 0], [61, 1], [60, 1], [59, 0], [58, 0], [57, 1], [56, 0], [55, 1], [54, 0], [53, 0], [52, 1], [51, 0], [50, 1], [49, 0], [48, 1], [47, 1], [46, 1], [45, 1], [44, 1], [43, 0], [42, 1], [41, 1], [40, 0], [39, 1], [38, 0], [37, 1], [36, 1], [35, 0], [

arrayS
[[1000, 0], [999, 0], [998, 1], [997, 1], [996, 0], [995, 0], [994, 1], [993, 0], [992, 1], [991, 0], [990, 1], [989, 0], [988, 1], [987, 1], [986, 0], [985, 1], [984, 1], [983, 1], [982, 0], [981, 1], [980, 0], [979, 0], [978, 0], [977, 0], [976, 1], [975, 0], [974, 0], [973, 0], [972, 1], [971, 0], [970, 1], [969, 1], [968, 0], [967, 1], [966, 1], [965, 1], [964, 0], [963, 1], [962, 0], [961, 0], [960, 0], [959, 1], [958, 0], [957, 0], [956, 1], [955, 1], [954, 0], [953, 0], [952, 0], [951, 0], [950, 0], [949, 1], [948, 0], [947, 0], [946, 1], [945, 0], [944, 1], [943, 1], [942, 1], [941, 1], [940, 0], [939, 1], [938, 0], [937, 1], [936, 0], [935, 0], [934, 1], [933, 1], [932, 0], [931, 1], [930, 1], [929, 0], [928, 1], [927, 1], [926, 0], [925, 1], [924, 1], [923, 0], [922, 1], [921, 0], [920, 1], [919, 0], [918, 0], [917, 0], [916, 0], [915, 0], [914, 0], [913, 1], [912, 1], [911, 1], [910, 1], [909, 1], [908, 0], [907, 1], [906, 0], [905, 0], [904, 1], [903, 1], [902, 0], [

## Question 5.2 (5 Points)

The following dataset has _irrelevant features_. Find them and use regularization to control them. 

Implement a training method that trains a logistic regression model and returns training and testing AUC values. This follows closely question 3.2 above. Next, fill in the driver code that trains the model for each regularization value and populates an array of training AUC, testing AUC, and weight vector norm values.

Hint: The weights for regularization parameter are displayed.

In [56]:
def train_logistic_regression(sgd, obj, w, train_set, test_set, num_epoch = 100, batch_size = 20)
  train_set1=train_set["data"]
  test_set1=test_set["data"]
  

  num_epoch.times do
    batch_of_examples=train_set1.each_slice(batch_size).to_a
    batch_of_examples.each do |batch|
      sgd.update(batch)
    end
  end
  
  train_sc=score_binary_classification_model(train_set1, sgd.weights, obj)
  test_sc=score_binary_classification_model(test_set1, sgd.weights, obj)

  train_auc= calc_auc_only(train_sc)
  test_auc=calc_auc_only(test_sc)
    
  return [train_auc, test_auc]
end


:train_logistic_regression

In [81]:
def test_logistic_regularizers(corner)
  stats = Hash.new {|h,k| h[k] = Array.new}
  [0.0, 0.5, 1.0, 5.0, 7.0, 10.0, 12.0, 15.0, 20.0, 25.0, 30.0, 35.0].each do |reg|
    tr_aucs = []
    te_aucs = []
    w_norms = []
    
    cross_validate corner, 5 do |tr, te, fold|
      w = Hash.new {|h,k| h[k] = 0.0}
      lr = 0.1
      obj = LogisticRegressionModelL2.new reg
      sgd = StochasticGradientDescent.new obj, w, lr
      train_auc, test_auc = train_logistic_regression(sgd, obj, w, tr, te, num_epoch = 30, batch_size = 200)

      tr_aucs << train_auc
      te_aucs << test_auc
      w_norms << norm(sgd.weights)
      puts "Reg: #{reg}", "Weights", sgd.weights if fold == 0 && reg < 0.5
    end
    stats[:reg] << reg
    stats[:tr_aucs] << mean(tr_aucs)
    stats[:tr_aucs_std] << stdev(tr_aucs)
    stats[:w_norms] << mean(w_norms)
    stats[:te_aucs] << mean(te_aucs)    
    stats[:te_aucs_std] << stdev(te_aucs)        
  end
  #puts stats
  return stats
end

:test_logistic_regularizers

In [82]:
def test_52()
  corner = z_normalize(read_sparse_data_from_csv("corner"))
  corner["data"].first
  
  stats = test_logistic_regularizers(corner)
  assert_true(stats[:tr_aucs].all? {|a| a > 0.7 and a < 1.0}, "1")
  assert_true(stats[:te_aucs].all? {|a| a > 0.7 and a < 1.0}, "2")
  assert_true(stats[:w_norms][0] > stats[:w_norms][6], "3")
  assert_true(stats[:w_norms][6] > stats[:w_norms].last, "4")    
  [Daru::DataFrame.new(stats), stats]
end

df, t_52_stats = test_52()
df

Reg: 0.0
Weights
{"x1"=>0.04787745074196486, "x2"=>-0.0347493895776463, "x3"=>-0.05419605483054221, "x4"=>0.010094358778060027, "x5"=>0.004332419320525407, "x6"=>0.0006062499259010844, "1"=>-0.7712414779248993}


,reg,tr_aucs,tr_aucs_std,w_norms,te_aucs,te_aucs_std
0,0.0,0.9777972015731209,0.003994294573487817,0.7750199752209566,0.9818531524812055,0.015131374745881979
1,0.5,0.9776658950525933,0.005122975386765284,0.5047565559294541,0.9807672396620177,0.016275827320231223
2,1.0,0.9772942812536689,0.005788539736584548,0.35522943592065376,0.9791360869922165,0.018218146131042896
3,5.0,0.9736141577974345,0.01139467953040912,0.09117529296307866,0.9751167684053195,0.025629965256903357
4,7.0,0.9734793177418795,0.011387683357007516,0.06601376284956802,0.975719057510777,0.02540226520539387
5,10.0,0.972129843771169,0.014371215032285037,0.046682151094450154,0.9741583661462634,0.028005584486912743
6,12.0,0.9715737169189447,0.015387923198923292,0.03905468516346392,0.9733396958825316,0.029669878507288902
7,15.0,0.9714743433306054,0.015616310487715121,0.03136466552282263,0.973548382027376,0.0297185360012829
8,20.0,0.9696820159714605,0.017867888973382776,0.023611373012081695,0.9716633449055271,0.03206516772802665
9,25.0,0.9686759502288561,0.01858216138418281,0.01892882646876263,0.9706775744981255,0.0326384571380885


### Interpretation

Running 5-fold cross validation for several parameter combinations, we can find the best regularization parameter based on the average of the test sets obtained during cross validation. We call them test sets, but they are subsets of the original dataset--just not provided to the trainer. The plot below will help you find the best value. 

In [83]:
t_52_plot = Nyaplot::Plot.new
t_52_plot.x_label("Regularization Parameter").y_label("Average CV AUC")
t_52_plot.add(:line, t_52_stats[:reg], t_52_stats[:te_aucs]).color(:red)
t_52_plot.show() 

#<Nyaplot::Frame:0x0000000002ce8518 @properties={:panes=>[#<Nyaplot::Plot:0x0000000002ceabd8 @properties={:diagrams=>[#<Nyaplot::Diagram:0x0000000002ce8bf8 @properties={:type=>:line, :options=>{:x=>"data0", :y=>"data1", :color=>:red}, :data=>"0602924f-4063-4be9-9693-753d92b4603b"}, @xrange=[0.0, 35.0], @yrange=[0.9674339059371473, 0.9818531524812055]>], :options=>{:x_label=>"Regularization Parameter", :y_label=>"Average CV AUC", :zoom=>true, :width=>700, :xrange=>[0.0, 35.0], :yrange=>[0.9674339059371473, 0.9818531524812055]}}>], :data=>{"0602924f-4063-4be9-9693-753d92b4603b"=>#<Nyaplot::DataFrame:0x0000000002cea3e0 @name="0602924f-4063-4be9-9693-753d92b4603b", @rows=[{:data0=>0.0, :data1=>0.9818531524812055}, {:data0=>0.5, :data1=>0.9807672396620177}, {:data0=>1.0, :data1=>0.9791360869922165}, {:data0=>5.0, :data1=>0.9751167684053195}, {:data0=>7.0, :data1=>0.975719057510777}, {:data0=>10.0, :data1=>0.9741583661462634}, {:data0=>12.0, :data1=>0.9733396958825316}, {:data0=>15.0, :data1=>0.973548382027376}, {:data0=>20.0, :data1=>0.9716633449055271}, {:data0=>25.0, :data1=>0.9706775744981255}, {:data0=>30.0, :data1=>0.9688012193129726}, {:data0=>35.0, :data1=>0.9674339059371473}]>}, :extension=>[]}>

## Question 5.3 (2 Points)

Make the function below return an array of feature names you think are irrelevant. If you think features ```"a", "b"``` are irrelevant, write your function below as follows:

```ruby
def guess_irrelevant_features()
  answer = ["a", "b"]
  return answer
end
```

**Hint**: What does L2 regularization do for irrelevant features?

In [75]:
def guess_irrelevant_features()

  answer=["x4", "x5", "x6"]
  return answer
end

:guess_irrelevant_features

In [76]:
corner = read_sparse_data_from_csv("corner")

t53_answer = guess_irrelevant_features()
assert_true(t53_answer.is_a?(Array))
assert_false(t53_answer.empty?)
assert_false((corner["features"] & t53_answer).empty?)

### Hidden Test ###


## Question 5.4 (3 Points)

Based on the plot above for different regularization parameters, which has the best AUC?  If your answer is, say 1.2345, write the following: 

```ruby
def pick_best_regularizer()
  answer = 1.2345
  return answer
end
```

In [77]:
def pick_best_regularizer()
  answer=5.0
  return answer
end

:pick_best_regularizer

In [78]:
### Hidden Test ###